In [65]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [72]:
df_1 = pd.read_csv('Data/train.csv')

In [73]:
df = df_1.copy ()

df [['Modelo_CPU', 'Velocidad_CPU']] = df['Cpu'].str.extract(r'(.+)\s+(\d+\.\d+GHz)')

df['Velocidad_CPU'] = df['Velocidad_CPU'].str.extract(r'(\d+\.\d+)').astype(float)

# Eliminar la columna original "Cpu"
df.drop(columns=['Cpu'], inplace=True)

# Imprimir las primeras filas para verificar

df = df.dropna()
# Imprimir las primeras filas para verificar
df.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 851 entries, 0 to 911
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                851 non-null    int64  
 1   laptop_ID         851 non-null    int64  
 2   Company           851 non-null    object 
 3   Product           851 non-null    object 
 4   TypeName          851 non-null    object 
 5   Inches            851 non-null    float64
 6   ScreenResolution  851 non-null    object 
 7   Ram               851 non-null    object 
 8   Memory            851 non-null    object 
 9   Gpu               851 non-null    object 
 10  OpSys             851 non-null    object 
 11  Weight            851 non-null    object 
 12  Price_euros       851 non-null    float64
 13  Modelo_CPU        851 non-null    object 
 14  Velocidad_CPU     851 non-null    float64
dtypes: float64(3), int64(2), object(10)
memory usage: 106.4+ KB


In [76]:


# Resto del código para dividir los datos, entrenar y evaluar los modelos...


df_subset = df[['Ram', 'Inches', 'Gpu', 'Modelo_CPU', 'Velocidad_CPU']]

# Procesar la columna 'Ram'
df_subset['Ram'] = df_subset['Ram'].str.replace('GB', '').astype(int)

# Codificar la variable categórica 'Gpu' con OneHotEncoding
onehot_encoder_gpu = OneHotEncoder(sparse_output=False)
gpu_onehot = onehot_encoder_gpu.fit_transform(df_subset[['Gpu']])

# Convertir el resultado de OneHotEncoding en un DataFrame y unirlo con el DataFrame original
gpu_df = pd.DataFrame(gpu_onehot, columns=onehot_encoder_gpu.get_feature_names_out(['Gpu']))
df_subset = pd.concat([df_subset, gpu_df], axis=1).drop(columns=['Gpu'])

# Codificar la variable categórica 'Modelo_CPU' con OneHotEncoding
onehot_encoder_cpu = OneHotEncoder(sparse_output=False)
cpu_onehot = onehot_encoder_cpu.fit_transform(df_subset[['Modelo_CPU']])

# Convertir el resultado de OneHotEncoding en un DataFrame y unirlo con el DataFrame original
cpu_df = pd.DataFrame(cpu_onehot, columns=onehot_encoder_cpu.get_feature_names_out(['Modelo_CPU']))
df_subset = pd.concat([df_subset, cpu_df], axis=1).drop(columns=['Modelo_CPU'])

# Escalar las variables 'Ram', 'Inches' y 'Velocidad_CPU'
scaler = StandardScaler()
df_subset[['Ram', 'Inches', 'Velocidad_CPU']] = scaler.fit_transform(df_subset[['Ram', 'Inches', 'Velocidad_CPU']])

# Eliminar filas con valores faltantes en la columna 'Price_euros'
df_subset.dropna(subset=['Price_euros'], inplace=True)

# Preparar los datos para la regresión lineal múltiple
X = df_subset
y = df['Price_euros']

# Resto del código para dividir los datos, entrenar y evaluar los modelos...

# Preparar los datos para la regresión lineal múltiple
#X = df_subset # Las variables independientes (Ram, Inches, Gpu, Modelo_CPU y Velocidad_CPU)
#y = df['Price_euros']                        # La variable dependiente

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Ridge
ridge_model = Ridge(alpha=1.0)  # Puedes ajustar el valor de alpha según sea necesario
ridge_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_ridge = ridge_model.predict(X_test)

# Evaluar el modelo de Ridge
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("Ridge Regression:")
print(f"Mean Squared Error: {mse_ridge}")
print(f"R² Score: {r2_ridge}")

# Crear y entrenar el modelo de Lasso
lasso_model = Lasso(alpha=1.0)  # Puedes ajustar el valor de alpha según sea necesario
lasso_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_lasso = lasso_model.predict(X_test)

# Evaluar el modelo de Lasso
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\nLasso Regression:")
print(f"Mean Squared Error: {mse_lasso}")
print(f"R² Score: {r2_lasso}")

C:\Users\pobla\AppData\Local\Temp\ipykernel_15920\2575767217.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Ram'] = df_subset['Ram'].str.replace('GB', '').astype(int)


KeyError: ['Price_euros']